<a id='setup'></a>
# Initial setup 


In [ ]:
# -- MAIN IMPORT

import pyVHR as vhr
import numpy as np

# Plotting: set 'colab' for Google Colaboratory, 'notebook' otherwise
vhr.plot.VisualizeParams.renderer = 'colab'  # or 'notebook'

In [ ]:
# -- LOAD A DATASET

dataset_name = 'lgi_ppgi'          # the name of the python class handling it 
video_DIR = '/var/datasets/VHR1/'  # dir containing videos
BVP_DIR = '/var/datasets/VHR1/'    # dir containing BVPs GT

dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)
allvideo = dataset.videoFilenames

# print the list of video names with the progressive index (idx)
for v in range(len(allvideo)):
  print(v, allvideo[v])

In [ ]:
# -- PARAMETER SETTING

wsize = 8          # seconds of video processed (with overlapping) for each estimate 
video_idx = 6      # index of the video to be processed
fname = dataset.getSigFilename(video_idx)
sigGT = dataset.readSigfile(fname)
test_bvp = sigGT.data
bpmGT, timesGT = sigGT.getBPM(wsize)
videoFileName = dataset.getVideoFilename(video_idx)
print('Video processed name: ', videoFileName)
fps = vhr.extraction.get_fps(videoFileName)
print('Video frame rate:     ',fps)

In [ ]:
# -- DISPLAY VIDEO FRAMES

vhr.plot.display_video(videoFileName)

# Skin extraction

Estract the skin by two methods (using the `SignalProcessing` class):
* **convex hull** on landmarks by [MediaPipe Face Mesh](https://google.github.io/mediapipe/solutions/face_mesh) (a face geometry solution that estimates 468 3D face landmarks in real-time)
* **face parsing** by the BiSeNet CNN (see [face-parsing.PyTorch](https://github.com/zllrunning/face-parsing.PyTorch))

Once the skin is selected, select how to process it: 

* **Patches**: small facial regions of skin  centered on landmarks (provides multiple estimators)
* **Holistic**: convex hull of patches or face parsing CNN (provides a single  estimator)

***Note***: `SignalProcessing` is powered by CUDA

In [ ]:
sig_extractor = vhr.extraction.SignalProcessing()
sig_extractor.display_cuda_device()
sig_extractor.choose_cuda_device(0)

Use convex hull or face parsing to extract the skin

In [ ]:
# CPU based
sig_extractor.set_skin_extractor(vhr.extraction.SkinExtractionConvexHull('CPU'))
#sig_extractor.set_skin_extractor(vhr.extraction.SkinExtractionFaceParsing('CPU'))

# GPU based
sig_extractor.set_skin_extractor(vhr.extraction.SkinExtractionConvexHull('GPU'))
#sig_extractor.set_skin_extractor(vhr.extraction.SkinExtractionFaceParsing('GPU'))

Choose a specific number of frames of the video to process... 

In [ ]:
# set the number of seconds (0 for all video)
seconds = 0
sig_extractor.set_total_frames(seconds*fps)

### Color-thresholding

**OPTIONAL**: Both signal extraction and skin extraction have a color-threshold filter for removing unwanted RGB colors. We can set the RGB threshold interval using theese classes:

In [ ]:
vhr.extraction.SkinProcessingParams.RGB_LOW_TH = 2
vhr.extraction.SkinProcessingParams.RGB_HIGH_TH = 254

vhr.extraction.SignalProcessingParams.RGB_LOW_TH = 2
vhr.extraction.SignalProcessingParams.RGB_HIGH_TH = 254

## Visualize skin and landmarks 

* To visualize skin processing intermediate results call `set_visualize_skin_and_landmarks` method.
* To retrieve any intermediate result call the methods `get_visualize_skin` and 
`get_visualize_patches`

<img src='https://github.com/giulianogrossi/imgs/blob/main/pyVHR/landmark_on_face.png?raw=true' width=300px >

In [ ]:
# -- SET VISUALIZATION MODE 
sig_extractor.set_visualize_skin_and_landmarks(
      visualize_skin=True, 
      visualize_landmarks=True, 
      visualize_landmarks_number=True, 
      visualize_patch=True)

In [ ]:
# -- DEFINE A LANDMARK SUBSET

# choose predefined...
landmarks = vhr.extraction.MagicLandmarks.cheek_left_top +\
                   vhr.extraction.MagicLandmarks.forehead_center +\
                   vhr.extraction.MagicLandmarks.forehoead_right +\
                   vhr.extraction.MagicLandmarks.cheek_right_top +\
                   vhr.extraction.MagicLandmarks.forehead_left +\
                   vhr.extraction.MagicLandmarks.nose 

# ... or sample the face by 100 equispaced landmarks
landmarks = [2, 3, 4, 5, 6, 8, 9, 10, 18, 21, 32, 35, 36, 43, 46, 47, 48, 50, 54, \
             58, 67, 68, 69, 71, 92, 93, 101, 103, 104, 108, 109, 116, 117, \
             118, 123, 132, 134, 135, 138, 139, 142, 148, 149, 150, 151, 152, 182, 187, 188, 193, 197, 201, 205, 206, 207, \
             210, 211, 212, 216, 234, 248, 251, 262, 265, 266, 273, 277, 278, 280, \
             284, 288, 297, 299, 322, 323, 330, 332, 333, 337, 338, 345, \
             346, 361, 363, 364, 367, 368, 371, 377, 379, 411, 412, 417, 421, 425, 426, 427, 430, 432, 436]



In [ ]:
# -- VISUALIZE LANDMARK SUBSET

print('Num landmarks: ', len(landmarks))
vhr.plot.visualize_landmarks_list(landmarks_list=landmarks)

In [ ]:
# -- SET THE LANDMARK LIST
sig_extractor.set_landmarks(landmarks)

# ROI processing and RGB computation

Choose how to extract the RGB signal from ROI:

* **Holistic** mean
* **Patches** mean

Patches are square (with a fixed edge for all) or rectangular (with xy_dimension for each region).

<img src='https://github.com/giulianogrossi/imgs/blob/main/pyVHR/CUDA.png?raw=true' width=70px >



## Holistic extraction

In [ ]:
# -- HOLISTIC EXTRACTION
hol_sig = sig_extractor.extract_holistic(videoFileName)
print('Size: (#frames, #landmarks, #channels) = ',hol_sig.shape)

In [ ]:
# -- INTERACTIVE VISUALIZATION OF PATCHES
visualize_skin_coll = sig_extractor.get_visualize_skin()
print('Number of frames processed: ',len(visualize_skin_coll))
vhr.plot.interactive_image_plot(visualize_skin_coll,1.0)

## Patches extraction

In [ ]:
# -- PATCHES EXTRACTION
sig_extractor.set_square_patches_side(30.0)
patch_sig = sig_extractor.extract_patches(videoFileName, "squares", "mean")
print('Size: (#frames, #landmarks, #channels) = ', patch_sig.shape)

In [ ]:
# -- INTERACTIVE VISUALIZATION OF PATCHES
visualize_patches_coll = sig_extractor.get_visualize_patches()
print('Number of frames processed: ',len(visualize_patches_coll))
vhr.plot.interactive_image_plot(visualize_patches_coll,1.0)

## Signal windowing

Windowing means to split a video into a set of strided and overlapped windows of frames. For each window the RGB signal is estracted by averaging over pixels in holistic (all skin pixels) or local (averaging on patches) fashion. Shapes are `(rgb_channels, #frames)` and `(#landmarks, rgb_channels, #frames)` respectively. 

### Holistic


In [ ]:
# -- WINDOWING OF RGB SIGNALS (HOLISTIC)
windowed_hol_sig, timesES = vhr.extraction.sig_windowing(hol_sig, wsize, 1, fps)
print('Num windows: ',len(windowed_hol_sig))
print('Num channels and window length: ', windowed_hol_sig[0].shape)

In [ ]:
# -- PLOT A WINDOW (randomly chosen)
wind = np.random.randint(0, len(windowed_hol_sig))   # window number
vhr.plot.visualize_windowed_sig(windowed_hol_sig, wind)

### Patches


In [ ]:
# -- WINDOWING OF RGB SIGNALS ON PATCHES 
windowed_patch_sig, timesES = vhr.extraction.sig_windowing(patch_sig, wsize, 1, fps)
print('Num windows: ',len(windowed_patch_sig))
print('Num channels and window length: ', windowed_patch_sig[0].shape)

In [ ]:
# -- PLOT A WINDOW (randomly chosen)
w = np.random.randint(0, len(windowed_patch_sig))  # window number
vhr.plot.visualize_windowed_sig(windowed_patch_sig, 18)

# Pre-filtering

The implemented (standard) filters are:

* `rgb_filter_ths`: color threshold filter that filters out signals that, in at least one frame of the window, are outside the rgb colors interval `[(LOW, LOW, LOW), (HIGH, HIGH, HIGH)]` where `LOW` is the dictionary parameter `RGB_LOW_TH`, and `HIGH` is `RGB_HIGH_TH` (we suggest to always use this filter before applying a BVP method)
* `detrend`: apply detrend to the signal
* `sg_detrend`: apply detrend to the signal, i.e. remove the low-frequency components with the low-pass filter developed by Savitzky-Golay
* `zscore`: apply z-score to the signal
* `BPfilter`: apply Butterworth band-pass filter to the signal

### Holistic

In [ ]:
# -- APPLY TRESHOLDING ON RGB COLORS (suggested)

filtered_windowed_hol_sig = vhr.BVP.apply_filter(windowed_hol_sig, vhr.BVP.rgb_filter_th, params={'RGB_LOW_TH': 0, 'RGB_HIGH_TH': 255})
print('Num windows: ', len(filtered_windowed_hol_sig))
print('Win size: (#signals, #channels, #frames) = ', filtered_windowed_hol_sig[0].shape)

In [ ]:
# -- SELECT THE FILTER CASCADE

filtered_windowed_hol_sig = vhr.BVP.apply_filter(filtered_windowed_hol_sig, vhr.BVP.BPfilter, params={'order':6,'minHz':0.65,'maxHz':4.0,'fps':fps})
#filtered_windowed_hol_sig = vhr.BVP.apply_filter(filtered_windowed_hol_sig, vhr.BVP.detrend)
#filtered_windowed_hol_sig = vhr.BVP.apply_filter(filtered_windowed_hol_sig, vhr.BVP.sg_detrend)
#filtered_windowed_hol_sig = vhr.BVP.apply_filter(filtered_windowed_hol_sig, vhr.BVP.zscore)
#filtered_windowed_hol_sig = vhr.BVP.apply_filter(filtered_windowed_hol_sig, vhr.BVP.zeromean)
print('Num windows: ', len(filtered_windowed_hol_sig))
print('Win size: (#signals, #channels, #frames) = ', filtered_windowed_hol_sig[0].shape)

In [ ]:
# -- PLOT A WINDOW (randomly chosen)
w = np.random.randint(0, len(windowed_hol_sig))  # window number
vhr.plot.visualize_windowed_sig(filtered_windowed_hol_sig, w)

### Patches

In [ ]:
# -- APPLY TRESHOLDING ON RGB COLORS (suggested)

filtered_windowed_patch_sig = vhr.BVP.apply_filter(windowed_patch_sig, vhr.BVP.rgb_filter_th, params={'RGB_LOW_TH': 0, 'RGB_HIGH_TH': 255})
print('Num windows: ', len(filtered_windowed_patch_sig))
print('Win size: (#landmarks, #channels, #frames) = ', filtered_windowed_patch_sig[0].shape)

In [ ]:
# -- SELECT THE FILTER CASCADE

filtered_windowed_patch_sig = vhr.BVP.apply_filter(filtered_windowed_patch_sig, vhr.BVP.BPfilter, params={'order':6,'minHz':0.65,'maxHz':4.0,'fps':fps})
#filtered_windowed_patch_sig = vhr.BVP.apply_filter(filtered_windowed_patch_sig, vhr.BVP.sg_detrend)
#filtered_windowed_patch_sig = vhr.BVP.apply_filter(filtered_windowed_patch_sig, vhr.BVP.detrend)
#filtered_windowed_patch_sig = vhr.BVP.apply_filter(filtered_windowed_patch_sig, vhr.BVP.zscore)
#filtered_windowed_patch_sig = vhr.BVP.apply_filter(filtered_windowed_patch_sig, vhr.BVP.zeromean)
print('Num windows: ', len(filtered_windowed_patch_sig))
print('Win size: (#landmarks, #frames, #channels) = ', filtered_windowed_patch_sig[0].shape)

In [ ]:
# -- PLOT A WINDOW (randomly chosen)

w = np.random.randint(0, len(windowed_patch_sig))  # window number
vhr.plot.visualize_windowed_sig(filtered_windowed_patch_sig, w)

#Method: BVP extraction

To extract the BVP signal call the function `RGB_sig_to_BVP` with the following parameters:


*   `filt_windowed_sig`: the list of windows
*   `fps`: frame rate
*   `device_type`: `cuda`, `cpu`, `torch`
*   `method`: method function that supports method_type device
*   params: dictionary of parameters needed by the method ( default is {}).

Methods implemented:
* device_type cuda: cupy_CHROM, POS, ...

***Note***: pyVHR contains many methods, but you can also use a custom method. Remember that it must accept a numpy.ndarray with shape (num_estimators, channels, num_frames) and return a numpy.ndarray with shape (num_estimators, num_frames)

<img src='https://github.com/giulianogrossi/imgs/blob/main/pyVHR/CUDA.png?raw=true' width=70px >



### Holistic

In [ ]:
# -- APPLY A METHOD TO EXTRACT BVP

from pyVHR.BVP import *

hol_bvps = RGB_sig_to_BVP(windowed_hol_sig, fps, device_type='cpu', method=cpu_CHROM)
#hol_bvps = RGB_sig_to_BVP(windowed_hol_sig, fps, device_type='cuda', method=cupy_CHROM)
#hol_bvps = RGB_sig_to_BVP(windowed_hol_sig, fps, device_type='torch', method=torch_CHROM)
#hol_bvps = RGB_sig_to_BVP(windowed_hol_sig, fps, device_type='cuda', method=cupy_POS, params={'fps':fps})
#hol_bvps = RGB_sig_to_BVP(windowed_hol_sig, fps, device_type='cpu', method=cpu_POS, params={'fps':fps})
#hol_bvps = RGB_sig_to_BVP(windowed_hol_sig, fps, device_type='cpu', method=cpu_LGI)
#hol_bvps = RGB_sig_to_BVP(windowed_hol_sig, fps, device_type='cpu', method=cpu_GREEN)
#hol_bvps = RGB_sig_to_BVP(windowed_hol_sig, fps, device_type='cpu', method=cpu_ICA, params={'component':'all_comp'})

print('Number of windows: ', len(hol_bvps))
print('Number of estimators and number of number of frames in a windows: ', hol_bvps[0].shape)

*bvps* is a list of length num_windows of numpy.ndarray with shape (num_estimators,num_frames)

In [ ]:
# -- PLOT A WINDOW (randomly chosen)

w = np.random.randint(0, len(windowed_hol_sig))  # window number
vhr.plot.visualize_BVPs(hol_bvps, w)

###Patches



In [ ]:
# -- APPLY A METHOD TO EXTRACT BVP

from pyVHR.BVP import *

patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_CHROM)
#patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cuda', method=cupy_CHROM)
#patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='torch', method=torch_CHROM)
#patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cuda', method=cupy_POS, params={'fps':fps})
#patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_POS, params={'fps':fps})
#patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_LGI)
#patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_GREEN)
#patch_bvps = RGB_sig_to_BVP(filtered_windowed_patch_sig, fps, device_type='cpu', method=cpu_ICA, params={'component':'all_comp'})

print('Number of windows: ', len(patch_bvps))
print('Number of estimators and number of number of frames in a windows: ', patch_bvps[0].shape)

In [ ]:
# -- PLOT A WINDOW (randomly chosen)
w = np.random.randint(0, len(windowed_patch_sig))  # window number
vhr.plot.visualize_BVPs(patch_bvps, w)

# Post Filtering

We can apply all the filters showed before also to the *BVP*. 

###Holistic


In [ ]:
# -- APPLY BPFILTER TO BVP WINDOWED 

hol_bvps = vhr.BVP.apply_filter(hol_bvps, BPfilter, params={'order':6,'minHz':0.65,'maxHz':4.0,'fps':fps})
print('Num windows: ', len(hol_bvps))

In [ ]:
# -- PLOT A WINDOW (randomly chosen)
wind = np.random.randint(0, len(windowed_hol_sig))  # window number
vhr.plot.visualize_BVPs(hol_bvps, wind)

###Patches


In [ ]:
# -- APPLY BPFILTER TO BVP WINDOWED PATCHES

patch_bvps = vhr.BVP.apply_filter(patch_bvps, BPfilter, params={'order':6,'minHz':0.65,'maxHz':4.0,'fps':fps})
print('Num windows: ', len(patch_bvps))
print('Win size: (#landmarks, #frames) = ', patch_bvps[0].shape)

In [ ]:
# -- PLOT A WINDOW (randomly chosen)

w = np.random.randint(0, len(windowed_patch_sig))  # window number
vhr.plot.visualize_BVPs(patch_bvps, w)

##BVP spectrum

BVP spectrum analysis via PSD for holistic and patches approaches.

###Holistic


In [ ]:
w = np.random.randint(0, len(windowed_hol_sig))  # window number
vhr.plot.visualize_BVPs_PSD(hol_bvps, w, fps)

###Patches


In [ ]:
wind = np.random.randint(0,len(patch_bvps))  # window number
vhr.plot.visualize_BVPs_PSD(patch_bvps, wind, fps)

# BMP estimation 

This function process all the windows and all the estimators (one for holistic and many for patches), and returns a list of numpyndarray with shape (num_estimators,).

<img src='https://github.com/giulianogrossi/imgs/blob/main/pyVHR/CUDA.png?raw=true' width=70px >


###Holistic


In [ ]:
# -- BPM ESTIMATION 

hol_bpmES = vhr.BPM.BVP_to_BPM(hol_bvps, fps)       # CPU version
#hol_bpmES = vhr.BPM.BVP_to_BPM_cuda(hol_bvps, fps)  # CUDA version

### Patches - medians

In [ ]:
# -- BPM ESTIMATION BY PATCHES
patch_bpmES = vhr.BPM.BVP_to_BPM(patch_bvps, fps)          # CPU version

#patch_bpmES = vhr.BPM.BVP_to_BPM_cuda(patch_bvps, fps)    # CUDA version

In [ ]:
# -- MEDIANS OF BPMS

patch_median_bpmES, MAD = vhr.BPM.multi_est_BPM_median(patch_bpmES)

In [ ]:
# -- VISUALIZE ALL BPMs AND MEDIANS
vhr.plot.visualize_multi_est_BPM_vs_BPMs_list([patch_bpmES, timesES], [[patch_median_bpmES, timesES, "medianES"],[bpmGT, timesGT, "GT"]])

### Patches - PSD clustering


In [ ]:
# -- BPM ESTIMATION BY PSD CUMUL
psd_bpmES = vhr.BPM.BVP_to_BPM_PSD_clustering(patch_bvps, fps)      # CPU version

# BPM vs GT ANALYSIS

Error computation and visualization 

###Holistic


In [ ]:

print(len(hol_bpmES), len(bpmGT), len(timesES), len(timesGT))

In [ ]:
# -- PRINT ERRORS USING METRICS: RMSE, MAE, MAX, PCC

from pyVHR.utils.errors import getErrors, printErrors, displayErrors

RMSE, MAE, MAX, PCC, CCC = getErrors(hol_bpmES, bpmGT, timesES, timesGT)
printErrors(RMSE, MAE, MAX, PCC, CCC)
displayErrors(hol_bpmES, bpmGT, timesES, timesGT)

###Patches - medians


In [ ]:
# -- PRINT ERRORS USING METRICS: RMSE, MAE, MAX, PCC

from pyVHR.utils.errors import getErrors, printErrors, displayErrors

RMSE, MAE, MAX, PCC, CCC = getErrors(patch_median_bpmES, bpmGT, timesES, timesGT)
printErrors(RMSE, MAE, MAX, PCC, CCC)
displayErrors(patch_median_bpmES, bpmGT, timesES, timesGT)

###Patches - PSD clustering

In [ ]:
# -- PRINT ERRORS USING METRICS: RMSE, MAE, MAX, PCC
from pyVHR.utils.errors import getErrors, printErrors, displayErrors
RMSE, MAE, MAX, PCC, CCC = getErrors(psd_bpmES, bpmGT, timesES, timesGT)
printErrors(RMSE, MAE, MAX, PCC, CCC)
displayErrors(psd_bpmES, bpmGT, timesES, timesGT)

In [ ]:
vhr.plot.visualize_BVPs_PSD_clutering(bpmGT, timesGT, patch_bvps, timesES, fps)